In [1]:
DATASET_PATH = '../data/raw/data_carrers.csv'
OUTPUT_PATH = '../data/processed/processed-dataset.csv'
OUTPUT_PATH_SMOTE = '../data/processed/processed-dataset_smnote.csv'
import pandas as pd
from transformers import pipeline
import spacy
import re
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings

# Cargar el modelo de SpaCy para español
nlp = spacy.load("es_core_news_sm")
warnings.filterwarnings("ignore")

# 1) Modelo para traducir de español a inglés
model_name_es_en = "Helsinki-NLP/opus-mt-es-en"
translator_es_en = pipeline(
    "translation_es_to_en",
    model=model_name_es_en,
    tokenizer=model_name_es_en
)

# 2) Modelo para traducir de inglés a español
model_name_en_es = "Helsinki-NLP/opus-mt-en-es"
translator_en_es = pipeline(
    "translation_en_to_es",
    model=model_name_en_es,
    tokenizer=model_name_en_es
)


from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = "tuner007/pegasus_paraphrase"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

def paraphrase_pegasus(input_text, num_return_sequences=2):
    batch = tokenizer(
        [input_text],
        truncation=True,        padding="longest",
        return_tensors="pt"
    )
    outputs = model.generate(
        **batch,
        max_length=128,
        num_beams=5,
        num_return_sequences=num_return_sequences,
        temperature=0.9,
        top_k=30,
        top_p=0.85,
        repetition_penalty=2.5
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)




def manual_back_translation(text, translator_es_en, translator_en_es):
    # 1) Traducir de español a inglés
    result_en = translator_es_en(text, max_length=512, truncation=True)
    # result_en es una lista de diccionarios [{'translation_text': "..."}]
    text_en = result_en[0]["translation_text"]

    # 2) Traducir la versión en inglés de vuelta a español
    result_es = translator_en_es(text_en, max_length=512, truncation=True)
    text_es = result_es[0]["translation_text"]

    return text_es


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Cargar el dataset
df = pd.read_csv(DATASET_PATH, encoding="UTF-8")

df_augmented_texts = []
df_augmented_labels = []
for idx, row in df.iterrows():
    original_text = row['TEXTO']
    label = row['CARRERA']

    for _ in range(1):
        # Ejecutar la traducción ida y vuelta
        back_translated = manual_back_translation(
            original_text,
            translator_es_en,
            translator_en_es
        )
        df_augmented_texts.append(back_translated)
        df_augmented_labels.append(label)
    # Genera, por ejemplo, 1 o 2 versiones back-translated
    # ¡Ojo! Esto puede ser lento si tu dataset es grande
    
    text = translator_es_en(original_text, max_length=512, truncation=True) 

    paraphrased_texts = paraphrase_pegasus(text[0]['translation_text'])

    for para_text in paraphrased_texts:
        df_augmented_texts.append(translator_en_es(para_text, max_length=512, truncation=True)[0]['translation_text'] )
        df_augmented_labels.append(label)
        

# Crear el df de ejemplos aumentados
df_aug = pd.DataFrame({
    'TEXTO': df_augmented_texts,
    'CARRERA': df_augmented_labels
})

# Concatenar con el original
df_final = pd.concat([df, df_aug]).reset_index(drop=True)


df = df_final


# Eliminar valores nulos y duplicados
df = df.dropna()
df = df.drop_duplicates()
# Preprocesar el texto
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\sáéíóúñü]', '', text)  # Eliminar caracteres especiales, pero conservar tildes
    text = re.sub(r'\s+', ' ', text).strip()  # Reemplaza múltiples espacios
    
    # Procesar el texto con SpaCy sin eliminar todas las stopwords
    doc = nlp(text)
    cleaned_text = " ".join([token.text for token in doc if not token.is_punct])
    
    return cleaned_text



df['TEXTO'] = df['TEXTO'].apply(preprocess_text)

# Mapear las categorías a índices numéricos
print(df['CARRERA'].unique())
categories = df['CARRERA'].unique().tolist()
category_to_index = {category: idx for idx, category in enumerate(categories)}
df['LABEL'] = df['CARRERA'].map(category_to_index)

################################SMOTE#######################

# Aplicar SMOTE para balancear las clases
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df['TEXTO'])

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, df['LABEL'])

df_balanced_texts = [" ".join(lista) for lista in vectorizer.inverse_transform(X_resampled)]
# Crear un nuevo DataFrame con los datos balanceados
df_balanced = pd.DataFrame({
    'TEXTO':df_balanced_texts,  # Convertir de nuevo a texto
    'LABEL': y_resampled
})

# Guardar el dataset balanceado
df_balanced.to_csv(OUTPUT_PATH_SMOTE, index=False)

# Mostrar la nueva distribución de clases
new_label_counts = df_balanced['LABEL'].value_counts()
print("Distribución de clases después de SMOTE:\n", new_label_counts)


df = df_balanced

######################$#

texts = df['TEXTO'].tolist()
labels = df['LABEL'].tolist()

df.to_csv(OUTPUT_PATH, index=False)

num_registros = df.shape[0]
print(f"Número total de registros en el dataset final: {num_registros}")

print(labels)

#######################GENERACION DE TABLA DE MUESTRAS ANTES Y DESPUES ###################################

import pandas as pd
from collections import Counter

# df_original: dataset antes de aplicar SMOTE
# df_balanced: dataset después de aplicar SMOTE 

# Crear DataFrame de conteo antes de SMOTE
before_counts = df_final['CARRERA'].value_counts().sort_index()
# Crear DataFrame de conteo después de SMOTE
after_counts = df_balanced['LABEL'].value_counts().sort_index()

# Mapeo inverso de índice a nombre de carrera
index_to_category = {v: k for k, v in category_to_index.items()}

# Construir DataFrame comparativo
summary_df = pd.DataFrame({
    "Categoría": [index_to_category[i] for i in after_counts.index],
    "Muestras Antes": before_counts.values,
    "Muestras Después": after_counts.values
})

print(summary_df)

